# Stacked Bar Explore

This notebook contains the base code for a stacked bar graph. 

In [1]:
import pandas as pd
import matplotlib
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models import ColumnDataSource, Div, HoverTool, LabelSet
from bokeh.models.glyphs import Ellipse

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)


In [2]:


text = ['Health', 'Education', 'Gender', 'Macro', 'Finance', 'PSD', 'Conflict', 'Taxation', 'Democracy']
# example factor:
f1 = np.array([100, 140, 350, 400, 203, 504, 605, 304, 405])
f2 = np.array([60, 530, 540, 330, 200, 406, 709, 403, 509])
f3 = np.array([700, 400, 400, 600 , 320, 670, 302, 609, 304])


# data frame 
df = pd.DataFrame({'text': text, 'f1': f1, 'f2': f2, 'f3': f3})

df.head()

,text,f1,f2,f3
0,Health,100,60,700
1,Education,140,530,400
2,Gender,350,540,400
3,Macro,400,330,600
4,Finance,203,200,320


In [5]:
def long(df, cat_var, value_vars):
    ''' make sure the dataset is in ESDB long format.'''
    # reorder the dataframe is necessary for the rename code to work properly 
    # the wide to long function needs a common begining to each column to be shifted
    
    
    df = df[[cat_var] + value_vars]
    
    # rename columns 
    df.columns = [cat_var] + ['f'+str(f) for f in range(0, len(df.columns[1:]))]
    
    # move from wide to long format
    l = pd.wide_to_long(df, stubnames='f', i=[cat_var], j='var').reset_index()

    
    
    return l

df_plot = long(df, 'text', ['f1', 'f2', 'f3'])
df_plot.head(10)

,text,var,f
0,Health,0,100
1,Education,0,140
2,Gender,0,350
3,Macro,0,400
4,Finance,0,203
5,PSD,0,504
6,Conflict,0,605
7,Taxation,0,304
8,Democracy,0,405
9,Health,1,60


In [10]:
def new_style(p): 
    p.legend.label_text_font_size = '11pt'
    p.legend.background_fill_alpha = 0.3
    p.grid.grid_line_alpha=0.2
    p.title.text_color = '#383951'
    p.title.text_font = 'Gill Sans MT'
    p.title.text_font_style = "bold"
    p.grid.grid_line_color='white'
    p.grid.grid_line_width=1.5
    p.axis.axis_label_text_font = 'Gill Sans MT'
    p.legend.label_text_font =  "Gill Sans Light"
    p.axis.axis_label_text_color = '#999999'
    p.axis.axis_label_text_font_style = 'normal'
    p.xaxis.minor_tick_line_color = None # turn off x-axis minor ticks
    p.yaxis.minor_tick_line_color = None # turn off y-axis minor ticks
    p.axis.axis_line_color=None
    p.axis.major_label_text_font_size = '12pt'
    p.axis.major_tick_line_color=None
    p.axis.minor_tick_line_color=None
    p.axis.major_label_text_color='#999999'
    p.outline_line_color = None
    p.axis.axis_label_text_font_size = '12pt'
    p.xaxis.axis_label = ''
    p.yaxis.axis_label = ''
    p.yaxis.visible = False  
    p.xaxis.visible = False
    p.title.text_font_size= '30pt'
    
    return p
    
    
    
palette = {'USAID Blue': '#002F6C', 'USAID Red': '#BA0C2F', 'Rich Black': '#212721', 'Medium Blue': '#0067B9',
    'Light Blue': '#A7C6ED', 'Dark Red': '#651D32', 'Dark Gray': '#6C6463', 'Medium Gray': '#8C8985', 'Light Gray': '#CFCDC9'}


### Define the Stacked Bar Function

In [49]:
def gen_stacked_bar(data, cat_var, stacked_var, value_var, colors = 'default', title_text='stacked bar', orientation='x-axis', 
                   alpha=0.8): 
    '''This function generates a stacked bar given data and three variables in long form. The 
    user can also choose whether the stack is vertical or horizontal in orientation. '''
    
    palette = {'USAID Blue': '#002F6C', 'Medium Blue': '#0067B9',  'Light Blue': '#A7C6ED', 'Medium Gray': '#8C8985', 'Light Gray': '#CFCDC9', 'USAID Red': '#BA0C2F', 'Rich Black': '#212721', 'Medium Blue': '#0067B9',
    'Light Blue': '#A7C6ED', 'Dark Red': '#651D32', 'Dark Gray': '#6C6463', 'Medium Gray': '#8C8985', 'Light Gray': '#CFCDC9'}
    
    # generate the lists for labels 
    bars=list(data[cat_var].unique())
    stacks = [str(i) for i in list(data[stacked_var].unique())]
    colors = list(palette.values())[0:len(stacks)]
    
    # generate the dictionary of data points 
    d = {'bars': bars}
    
    for i in stacks: 
        d[i] = list(data[data[stacked_var]==int(i)][value_var].values)
    
    
    
    # generate the plot 
    p = figure(x_range=bars, plot_height=250, title='',
           toolbar_location=None, tools="hover")
    
    if orientation=='x-axis' : 
        p.vbar_stack(stacks, x='bars', width=0.9, color=colors, source=d)
    
    p = new_style(p)
    
    p.yaxis.visible = True  
    p.xaxis.visible = True
    
    return p

show(gen_stacked_bar(df_plot, 'text', 'var', 'f'))

In [39]:
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.plotting import figure

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ["2015", "2016", "2017"]
colors = ["#c9d9d3", "#718dbf", "#e84d60"]

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 4, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

p = figure(x_range=fruits, plot_height=250, title="Fruit Counts by Year",
           toolbar_location=None, tools="hover", tooltips="$name @fruits: @$name")

p.vbar_stack(years, x='fruits', width=0.9, color=colors, source=d,
             legend=[value(x) for x in years])

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)

NameError: name 'd' is not defined

In [16]:


def radar_plot(df, obs_var, cat_var, value_var, title_text = 'Radar Plot'): 


    output_notebook()

    ###########################################
    ### data input 
    ###########################################
    
    # find min and max values
    max_var = df[value_var].max()
    min_var = df[value_var].min()
    
    

    
    # generate tick marks
    l = matplotlib.ticker.AutoLocator()
    l.create_dummy_axis()
    
    
    axis_values = l.tick_values(min_var, max_var)
    # scale each value by the second to last value for plotting 
    axis_scaled = np.array(axis_values)/axis_values[-2]
    
        
    # generate dictionary of all 
    reals = {}
    scales = {}
    for i in list(df[obs_var].unique()):
        
        reals[str(i)] = df[df[obs_var]==i][value_var].values
        #scale by the largest value in the axis generated by matplotlib
        scales[str(i)] = df[df[obs_var]==i][value_var].values/axis_values[-2]/2
        
        
        
        
    # address if the tick values are too many
    length = len(axis_values) 
    if length>6: 
        ## access every other observatation beginning from the second to last observation moving backwards
        ### add the extra value for consistancy in the labelling from [-1] no matter the length
        axis_values = list(reversed([axis_values[-2+(-2*i)] for i in range(0, int(np.round(len(axis_values)/2)))]))+[1]
        axis_scaled = list(reversed([axis_scaled[-2+(-2*i)] for i in range(0, int(np.round(len(axis_scaled)/2)))]))+[1]
    

    
    

    

    # set values to be ploted
    f1 = scales['0']
    f2 = scales['1']
    f3 = scales['2']
    flist = [f1,f2,f3]
    
    # list of names
    text = list(df[cat_var].unique())



    



    ## generate the number of vars to generate the shape. 
    num_vars = len(text)

    ##############################################
    ######## Functions 

    theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)
    # rotate theta such that the first axis is at the top
    theta += np.pi/2

    def unit_poly_verts(theta, r):
        """Return vertices of polygon for subplot axes.
        This polygon is circumscribed by a unit circle centered at (0.5, 0.5)
        """
        x0, y0, r = [0.5, 0.5, r]
        verts = [(r*np.cos(t) + x0, r*np.sin(t) + y0) for t in theta]
        return verts
    
    # turn data into circlular options
    def radar_patch(r, theta):
        yt = (r) * np.sin(theta) + 0.5
        xt = (r) * np.cos(theta) + 0.5
        return xt, yt
    
    
    
    ######### Generate plot
    
    p = figure(title="", plot_width = 740, plot_height= 760, x_range=(-.23,1.2), y_range=(-0.23,1.2), 
               tools='save,tap') 
    
    


    #################################
    #### Generate labels 
    #################################
    
    # generate the locations of the labels 
    verts = unit_poly_verts(theta, 0.55)
    x = [v[0] for v in verts] 
    y = [v[1] for v in verts]
    
    # separate between right and left side vars 
    a = (int(np.round(len(text)/2)))+1
    left_labels = text[:a]
    right_labels = text[a:]

    # plot right and left labels (difference -> text align)
    source_left = ColumnDataSource({'x':x[:a],'y':y[:a],'text':left_labels})
    source_right = ColumnDataSource({'x': x[a:]+[0.5],'y':y[a:],'text':right_labels})
    
    label_left = LabelSet(x="x",y="y",text="text",source=source_left, text_font='Gill Sans MT', text_font_size='15pt', 
                     text_color='grey', text_align = 'right')
    label_right = LabelSet(x="x",y="y",text="text",source=source_right, text_font='Gill Sans MT', text_font_size='15pt', 
                     text_color='grey')
    # add to plot
    p.add_layout(label_left)
    p.add_layout(label_right)
    
    
    
    
    
    
    #################################
    # generate background 
    #################################
    # circles

    for i in axis_scaled:
        glyph = Ellipse(x=0.5, y=0.5, width=i, height=i, fill_color=None, line_color="lightgrey", line_alpha=0.5) 
        p.add_glyph(glyph)
    
    #lines - generate coordinates - lines from center to coordinates 
    verts = unit_poly_verts(theta, 0.50)
    x_lines = [v[0] for v in verts] 
    y_lines = [v[1] for v in verts]
    
    for i in range(0,len(x_lines)): 
        p.line(x=(0.5, x_lines[i]), y=(0.5, y_lines[i]), line_width=3, line_color="lightgrey", line_alpha=0.5)
    
    #### numbered  
    nums = axis_values[:-1]
    x = np.array(axis_scaled)/2
    x = [.5-i for i in  np.array(axis_scaled)/2]
    y =[0.5, 0.5, 0.5, 0.5, 0.5]

    source = ColumnDataSource({'x':x,'y':y,'text':nums})

    numbers = LabelSet(x="x",y="y",text="text",source=source, text_font='Gill Sans MT', text_font_size='10pt', 
                     text_color='grey')

    p.add_layout(numbers)
    
    
    
    
    ##################################
    ####### Plot Patches and circles 
    ###################################
    
    # generate the unique observation variables 
    obs = list(df[obs_var].astype('str').unique())
    
    
    # this sets a maximum numbe of observations at six which is reasonable
    colors = [palette[i] for i in ['USAID Blue', 'USAID Red', 'Medium Blue', 'Light Blue', 'Dark Red', 'Medium Gray']]
    
    sources1 = pd.DataFrame()
    
    ##### Patches
    for i in range(len(flist)):
        xt, yt = radar_patch(flist[i], theta)

        sources1 = sources1.append(pd.DataFrame({'xt': [xt], 'yt': [yt], 'obs': obs[i], 'colors':colors[i]}))
    

    
    
    r = p.patches(xs='xt', ys='yt', fill_alpha=0.15, line_alpha = .5,color='colors', line_width = 5, legend='obs', 
              source = ColumnDataSource(sources1), hover_line_color=palette['Dark Gray'],hover_color='colors',
                 hover_fill_alpha=.15)
    hover_p = HoverTool(
            renderers=[r],
            tooltips="""
        <div> 

            <div>
                <span style="font-size: 15px; font-family: 'Gill Sans MT'; color: black;"><b>Observation:</b> @obs</span>
            </div>
        </div>
    """

    )
    p.add_tools(hover_p)              


    #### CIRCLE Graph (eventually change to be one source file)

    sources = {}
    for i in range(0, len(obs)): 
        xt, yt = radar_patch(flist[i], theta)
        sources[i] = {'Category': text, 'Obs': [i]*len(text), 'Value': reals[obs[i]], 'yt': yt, 'xt': xt}

        
    ########## tooltip settings 
        
    TOOLTIPS = """
        <div> 
            <div>
                <span style="font-size: 15px; font-family: 'Gill Sans MT'; color: black; "><b>Observation:</b> @Obs</span>

            </div>
            <div>
                <span style="font-size: 15px; font-family: 'Gill Sans MT';color: black;"><b>Category:</b> @Category</span>
            </div>
            <div>
                <span style="font-size: 15px; font-family: 'Gill Sans MT'; color: black;"><b>Value:</b> @Value</span>
            </div>
        </div>
    """


    for i in range(0, len(obs)): 
        s = p.circle(x='xt', y='yt', color = colors[i], source=sources[i],size = 10, fill_alpha=.6, 
                    hover_line_color='black', hover_color=colors[i])
        hover_circle = HoverTool(
                renderers=[s],
                tooltips=TOOLTIPS
        )
        p.add_tools(hover_circle)



    
    ################################
    ######## HOVER
    ################################


    
    #p.tools = None
    #############################
    #### LEGENDs and STYLE
    ##############################
    
    
    p.legend.location = 'bottom_right'
    p.legend.orientation = 'horizontal'

    p.title.text = title_text
    
    p = new_style(p)

    show(p)
    
    return p

p = radar_plot(df_plot, 'var', 'text', 'f')

Loading BokehJS ...